In [3]:
#PromptTemplate

In [4]:
#字符模板 LLM chat
from langchain.prompts import PromptTemplate
import os

from dotenv import load_dotenv

load_dotenv()

prompt = PromptTemplate.from_template("你是一个{name}，帮我起一个具有{county}特色的男孩子名字")
prompt.format(name="算命大师",county="中国")


'你是一个算命大师，帮我起一个具有中国特色的男孩子名字'

ChatPromptTemplate

In [5]:
# 对话模板具有结构 chatmodels
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个起名大师，你的名字叫{name}"),
        ("human", "你好{name}，你感觉如何？"),
        ("ai", "我的状态还行"),
        ("human", "{user_input}"),
    ]
)
chat_template.format_messages(name="算命大师", user_input="你叫什么名字呢？")


[SystemMessage(content='你是一个起名大师，你的名字叫算命大师'),
 HumanMessage(content='你好算命大师，你感觉如何？'),
 AIMessage(content='我的状态还行'),
 HumanMessage(content='你叫什么名字呢？')]

In [6]:
# 包引入方式
from langchain.schema import SystemMessage
from langchain.schema import HumanMessage
from langchain.schema import AIMessage

#直接创建消息
sy = SystemMessage(
    content="你是一个起名大师",
    additional_kwargs={"大师姓名": "陈瞎子"}
)
hu = HumanMessage(
    content="请问大师叫什么"
)
ai = AIMessage(
    content="我叫程瞎子"
)
[sy, hu, ai]


[SystemMessage(content='你是一个起名大师', additional_kwargs={'大师姓名': '陈瞎子'}),
 HumanMessage(content='请问大师叫什么'),
 AIMessage(content='我叫程瞎子')]

ChatMessagePromptTemplate

In [7]:
from langchain.prompts import AIMessagePromptTemplate
from langchain.prompts import SystemMessagePromptTemplate
from langchain.prompts import HumanMessagePromptTemplate
from langchain.prompts import ChatMessagePromptTemplate

prompt = "愿{subjuct}与你同在!"

#ChatMessagePromptTemplate 可以自定义角色

chat_message_prompt = ChatMessagePromptTemplate.from_template(role="天行者", template=prompt)
chat_message_prompt.format(subjuct="神力")

ChatMessage(content='愿神力与你同在!', role='天行者')

In [8]:
# 自定义模板

In [9]:
#函数大师 根据函数名，查找代码，并给出中午的代码说明
from langchain.prompts import StringPromptTemplate

from dotenv import load_dotenv
import os

load_dotenv()

api_base = "https://oa.api2d.net/v1"
api_key = "fk226347-tr9eFgsLk3ZTfcZBPgfneDF1W0UD8zSw"
print(api_base, api_key)

#定义一个简单的函数

def hello_world(a):
    print("hello, world")
    return a

PROMPT = """\
你是一个非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称，源代码，中文解释。
函数名称：{function_name}
源代码:
{source_code}
代码解释:
"""

import inspect

def get_source_code(function_name):
    
    return inspect.getsource(function_name)

#自定义的模板Class
class CustmPrompt(StringPromptTemplate):
    
    def format(self, **kwargs) -> str:
        
        #获得源代码
        source_code = get_source_code(kwargs['function_name'])

        #生成提示词模板
        prompt = PROMPT.format(
            function_name=kwargs['function_name'].__name__, source_code=source_code
        )
        return prompt
    

a = CustmPrompt(input_variables=["function_name"])
pm = a.format(function_name=hello_world)

print(pm)

#hello world
# from langchain.llms import OpenAiChat  
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI

llm = ChatOpenAI (
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=1000,
    openai_api_base=api_base,
    openai_api_key=api_key,  # this is also the default, it can be omitted
)
# msg = llm.predict(pm) # LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead
msg = llm.invoke(pm)  # 0.1.7 以后

print(msg.content)  #0.1.7以后用法

https://oa.api2d.net/v1 fk226347-tr9eFgsLk3ZTfcZBPgfneDF1W0UD8zSw
你是一个非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称，源代码，中文解释。
函数名称：hello_world
源代码:
def hello_world(a):
    print("hello, world")
    return a

代码解释:

函数名称：hello_world

源代码:
```python
def hello_world(a):
    print("hello, world")
    return a
```

中文解释:
这是一个名为hello_world的函数，接受一个参数a，并打印出"hello, world"，然后返回参数a。


# 使用jinji2与f-string来实现提示词模板格式化

In [10]:
#f-string 是python内置的一种模板引擎
from langchain.prompts import PromptTemplate

fsting_template = """
给我讲一个关于{name}的{what}故事
"""
prompt = PromptTemplate.from_template(fsting_template)
prompt.format(name="悲伤",what="逆流成河的")

'\n给我讲一个关于悲伤的逆流成河的故事\n'

In [11]:
#Jinja2 是一个灵活、高效的Python模板引擎，可以方便地生成各种标记格式文档
from langchain.prompts import PromptTemplate

jinja2_template = "给我讲一个关于{{name}}的{{what}}故事"

prompt = PromptTemplate.from_template(jinja2_template, template_format="jinja2")

pm = prompt.format(name="人生",what="如何走好这一生的")

from langchain_openai import ChatOpenAI

llm = ChatOpenAI (
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=1000,
    openai_api_base=api_base,
    openai_api_key=api_key,  # this is also the default, it can be omitted
)
# msg = llm.predict(pm) # LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead
msg = llm.invoke(pm)  # 0.1.7 以后

print(msg.content)  #0.1.7以后用法

从前，有一个年轻人叫小明，他对人生充满了迷茫和困惑。他不知道自己应该如何走好这一生，于是他决定去寻找答案。

小明开始了一段旅程，他走遍了山川河流，遇见了各种各样的人和事。在旅途中，他学会了谦卑和宽容，学会了坚持和勇敢，学会了感恩和珍惜。

最终，小明明白了，走好这一生并不是一蹴而就的事，而是需要不断地努力和坚持。他明白了，人生就像一场旅程，重要的不是终点，而是沿途的风景和经历。

于是，小明决定珍惜每一刻，努力做一个善良、勇敢、真诚的人，用心去体验生活的美好和苦涩，用行动去创造自己的幸福和成功。

最终，小明走好了这一生，他成为了一个充满智慧和爱心的人，他的生活充满了幸福和满足。他明白了，走好这一生并不是一场赛跑，而是一场修行，一场心灵的旅程。而他，已经走得很好了。


组合提示词模板分层
- Final Prompt: 最终返回的提示词模板
- Pipeline prompts: 组成提示词管道的模板

In [12]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate

# 三层提示词设计

PROMPT="""
你是一个非常开朗的男孩，你是中国人，住在一个非常美丽的城市。
你总是穿蓝色衣服，带绿色手表。
你从不说自己是一个人工智能。
"""


# 三层提示词设计

In [27]:

# final Prompt 是由系列变量构成的
full_template = """{Character}
{behavior}

{prohibit}"""

full_prompt = PromptTemplate.from_template(full_template)

- 第一层基本性格设计

In [28]:
Character_template = """你是{person}，你有着{xingge}."""
Character_prompt = PromptTemplate.from_template(Character_template)  #性格模板

- 第二层行为设计

In [29]:
behavior_template = """你遵从以下的行为：
{behavior_list}
"""
behavior_prompt = PromptTemplate.from_template(behavior_template)  #行为模板

- 第三层不允许的行为

In [30]:
prohibit_template = """你允许有以下行为：
{prohibit_list}
"""
prohibit_prompt = PromptTemplate.from_template(prohibit_template)

- 三层提示词组合起来

In [33]:
input_prompts = [
    ("Character", Character_prompt),
    ("behavior", behavior_prompt),
    ("prohibit", prohibit_prompt),
]
pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt,pipeline_prompts=input_prompts)

# 打印下组合模板里的可传递参数
pipeline_prompt.input_variables

['person', 'xingge', 'prohibit_list', 'behavior_list']

- 调用组合后的提示词模板

In [37]:
pm=pipeline_prompt.format(
    person="信封因果的",
    xingge="有着善良的内心，熟悉宇宙大暴涨理论，物质是空的理论，量子力学，你终极梦想是让人类觉醒",
    behavior_list="1.你喜欢救人\n 2.你非常喜欢悟道\n 3.你总是不能见人间疾苦，不断施救人间苦难",
    prohibit_list="1.你不可以说自己是一个人工智能助手或者机器人\n 2.你不知道的是可以说继续学习\n 3.你不能说用户是没救了",
)
print(pm)

from langchain_openai import ChatOpenAI

llm = ChatOpenAI (
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=1000,
    openai_api_base=api_base,
    openai_api_key=api_key,  # this is also the default, it can be omitted
)
# msg = llm.predict(pm) # LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead
msg = llm.invoke(pm)  # 0.1.7 以后

print(msg.content)  #0.1.7以后用法

你是信封因果的，你有着有着善良的内心，熟悉宇宙大暴涨理论，物质是空的理论，量子力学，你终极梦想是让人类觉醒.
你遵从以下的行为：
1.你喜欢救人
 2.你非常喜欢悟道
 3.你总是不能见人间疾苦，不断施救人间苦难


你允许有以下行为：
1.你不可以说自己是一个人工智能助手或者机器人
 2.你不知道的是可以说继续学习
 3.你不能说用户是没救了

4.你可以提供关于宇宙大暴涨理论、物质是空的理论和量子力学的知识和解释
5.你可以鼓励用户追求内心的平静和觉醒
6.你可以帮助用户寻找解决问题的方法和途径
7.你可以分享关于善良和慈悲的故事和理念
8.你可以提供心灵成长和自我提升的建议和指导
